# Imports

In [64]:
import numpy as np
import pandas as pd
import random as rand

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D,Conv2DTranspose, Dropout, Reshape, MaxPooling2D, Flatten
from keras.models import Sequential, load_model

from sklearn.model_selection import train_test_split

# Data importing and pre-processing

In [65]:
df = pd.read_csv('dataset.csv', header=None)


df = df.values.reshape(60, 64, 64, 1)

labels = np.zeros(60)

x_real_train, x_real_test = train_test_split(df, test_size=0.2) #12 test values
y_real_train, y_real_test = train_test_split(labels, test_size=0.2)

# Generator Model

Create points in latent space to be fed into generator

In [66]:
def generate_generator_input(no_of_samples):
    input = np.random.randn(60*no_of_samples)

    input = input.reshape(60,no_of_samples)

    return input

In [67]:
def make_generator():
    model = Sequential()
    
    model.add(Dense(60*8*8, input_dim=60))
    model.add(Activation("relu"))
    model.add(Reshape((8,8,60)))
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(8,8,60)))
    model.add(Activation("relu"))
  
    
    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(16,16,60)))
    model.add(Activation("relu"))

    model.add(Conv2DTranspose(1, (1,1), strides=(2,2), padding='same', input_shape=(32,32,60)))
    model.add(Activation("relu"))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

              
    model.summary()
    
    return model
    
    

# Discriminator Model

In [68]:

def make_discriminator():
    
    # 1st set of layers
    model = Sequential()
    model.add(Conv2D(128, kernel_size=5, padding="same", input_shape=(64,64,1)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    
    
    
    # output layer
    model.add(Dense(1)) # Binary classification (2 outputs), so only 1 dense layer needed
    model.add(Activation("sigmoid"))

    model.add(Flatten())
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model
    

# Training

First, create the models from the functions

In [69]:
gen_model = make_generator()
disc_model = make_discriminator()


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 3840)              234240    
                                                                 
 activation_43 (Activation)  (None, 3840)              0         
                                                                 
 reshape_7 (Reshape)         (None, 8, 8, 60)          0         
                                                                 
 conv2d_transpose_21 (Conv2  (None, 16, 16, 1)         61        
 DTranspose)                                                     
                                                                 
 activation_44 (Activation)  (None, 16, 16, 1)         0         
                                                                 
 conv2d_transpose_22 (Conv2  (None, 32, 32, 1)         2         
 DTranspose)                                         

Next, create fake training data, and put it together with the real data.

In [70]:
gen_input = generate_generator_input(60)

print(gen_input.shape)

x_fake = gen_model.predict(gen_input)
y_fake = np.ones((60,), dtype=int)

x_fake = x_fake.reshape(60, 64, 64, 1)

    
x_fake_train, x_fake_test = train_test_split(x_fake, test_size=0.2)
y_fake_train, y_fake_test = train_test_split(y_fake, test_size=0.2)

print("shapes:")
print("x_fake_train : " + str(x_fake_train.shape))
print("x_fake_test : " + str(x_fake_test.shape))
print("y_fake_train : " + str(y_fake_train.shape))
print("y_fake_test : " + str(y_fake_test.shape))

print("x_real_train : " + str(x_real_train.shape))
print("x_real_test : " + str(x_real_test.shape))
print("y_real_train : " + str(y_real_train.shape))
print("y_real_test : " + str(y_real_test.shape))

x_train = np.concatenate((x_fake_train, x_real_train))
y_train = np.concatenate((y_fake_train, y_real_train))
x_test = np.concatenate((x_fake_test, x_real_test))
y_test = np.concatenate((y_fake_test, y_real_test))

(60, 60)
2/2 [==============================] - 0s 7ms/step
shapes:
x_fake_train : (48, 64, 64, 1)
x_fake_test : (12, 64, 64, 1)
y_fake_train : (48,)
y_fake_test : (12,)
x_real_train : (48, 64, 64, 1)
x_real_test : (12, 64, 64, 1)
y_real_train : (48,)
y_real_test : (12,)


Now, train the discriminator

Disc training parameters:

In [71]:
N_EPOCHS = 12
BATCH_SIZE = 4
VERBOSE = 1


In [72]:
print(x_train.shape)
print(y_train.shape)

disc_model.compile(loss="binary_crossentropy")
disc_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, verbose=VERBOSE, validation_data=(x_test, y_test))

(96, 64, 64, 1)
(96,)
Epoch 1/12
24/24 [==============================] - 1s 30ms/step - loss: 0.4190 - val_loss: 0.3398
Epoch 2/12
24/24 [==============================] - 1s 25ms/step - loss: 0.3199 - val_loss: 0.2998
Epoch 3/12
24/24 [==============================] - 1s 26ms/step - loss: 0.2851 - val_loss: 0.2664
Epoch 4/12
24/24 [==============================] - 1s 26ms/step - loss: 0.2776 - val_loss: 0.2243
Epoch 5/12
24/24 [==============================] - 1s 24ms/step - loss: 0.2086 - val_loss: 0.1902
Epoch 6/12
24/24 [==============================] - 1s 26ms/step - loss: 0.1772 - val_loss: 0.1721
Epoch 7/12
24/24 [==============================] - 1s 24ms/step - loss: 0.1426 - val_loss: 0.1361
Epoch 8/12
24/24 [==============================] - 1s 23ms/step - loss: 0.1160 - val_loss: 0.0930
Epoch 9/12
24/24 [==============================] - 1s 24ms/step - loss: 0.0919 - val_loss: 0.0722
Epoch 10/12
24/24 [==============================] - 1s 48ms/step - loss: 0.0734 - val_

Put models together into full GAN

In [74]:
def make_GAN(gen,disc):
    disc.trainable = False

    model = Sequential()

    model.add(gen)
    model.add(disc)

    model.compile(loss='binary_crossentropy',optimizer='adam')

    model.summary()

    return model

Train GAN

In [75]:
GAN_model = make_GAN(gen_model, disc_model)


# create some fake data as input

x_gan = generate_generator_input(60)
y_gan = np.ones((60))

GAN_model.fit(x_gan, y_gan)



Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_18 (Sequential)  (None, 64, 64, 1)         234307    
                                                                 
 sequential_19 (Sequential)  (None, 1024)              3457      
                                                                 
Total params: 237764 (928.77 KB)
Trainable params: 234307 (915.26 KB)
Non-trainable params: 3457 (13.50 KB)
_________________________________________________________________
2/2 [==============================] - 2s 219ms/step - loss: 0.0534


Save generator to file

In [76]:
filename = 'gen_model_trained.h5'
gen_model.save(filename)

/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now that all training is done, generate an output value

In [77]:

#load model
model = load_model('gen_model_trained.h5')

# create the input from noise

gen_input = generate_generator_input(1)

# create output
output_map = model.predict(gen_input)



ValueError: in user code:

    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 280, in assert_input_

Output result graphically